### 네이버 뉴스 정보 가져오기

### 학습 내용 
 * 네이버 뉴스 여러개 단어 검색
 * 페이지별 언론사, 뉴스기사, URL

### 01. 웹 브라우저 띄우기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

### 네이버 뉴스 검색

In [2]:
driver = webdriver.Chrome('chromedriver_91')

url = 'https://search.naver.com/search.naver?query=apple&where=news'
driver.get(url)

import time
time.sleep(3)  # 홈페이지 로딩 시간 3초

In [23]:
# keyword_multi = ['이강덕 포항시장 백신', '태백시 류태호 백신', '허석 순천시장 백신', 
#                  '박영선 백신', '이강덕 시장 백신', '고3 백신', '최대호 안양시장 백신']

keyword_multi = ['고3 백신 접종']
                 
pages = 10

In [24]:
### 뉴스 검색 : //*[@id="nx_query"]
### 검색 버튼 : //*[@id="lnb.searchForm"]/fieldset/button

In [25]:
from datetime import datetime

now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)
print(now.day - 1)
now_day = now.day
now_hour = now.hour
file_make_time

25


'2021_1026_14'

In [26]:
import re
from datetime import timedelta

In [27]:
def date_cal(now_time):
    now = datetime.now()

    # 시간이 포함되어 있고,
    chk_time = now_time.find("시간")
    chk_day = now_time.find("일")
    if chk_time>=0: 
        hour = int( re.findall("\d+", now_time)[0] )
        
        # 현재 시간 - 적힌 시간이 음수가 나오면 현재 날짜에 -1을 해 준다. 
        cal_hour = now.hour - hour
        # print(cal_hour)
        if cal_hour < 0:
            news_date = timedelta(days = 1)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
        else:
            news_date = timedelta(days = 0)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
            
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        print("뉴스 날짜 : ", cal_date_time)
    
    # 일이 포함되어 있고,
    elif chk_day>=0: 
        news_day = int( re.findall("\d+", now_time)[0] )
        # print(news_day)
        
        # 현재 날짜 -1일전을 계산한다.
        news_date = timedelta(days = news_day)
        cal_now = now - news_date
        
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        # print("뉴스 날짜 : ", cal_date_time)
    else:
        cal_date_time = now_time
    return cal_date_time    

In [28]:
# date_cal("3일 전")

In [29]:
def keyword_news_n(keyword1, now_time):
    now_page_num = 0
    all_company1 = []
    all_article1 = []
    all_link1 = []
    all_date1 = []
    
    for i in range(1,pages,1):
        # 페이지 부분 정보 획득
        sel_num = driver.find_element_by_class_name("sc_page_inner")
        sel_all = sel_num.find_elements_by_class_name("btn")
        
        # 화면에 보이는 페이지 확인
        print("현재 페이지 : ", now_page_num+1, "키워드 : ", keyword1, "작성시간: ", now_time)
        len_all = len(sel_all)
        
        # 페이지 번호 확인 후, 해당되는 버튼 클릭
        for i in range(1, len_all+1 ):
            tmp_pre = '//*[@id="main_pack"]/div[2]/div/div/a[' + str(i) + ']'
            sel_num = driver.find_element_by_xpath(tmp_pre)
            sel_page_num = int(sel_num.text)
            if sel_page_num == (now_page_num+1):
                # 정보를 가져와서 클릭
                # 현재 페이지 입력
                sel_num.click()
                time.sleep(1)
                now_page_num += 1 
                print(now_page_num)
                break

        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        all_ul = soup.find("ul", class_="list_news").find_all("li", class_="bx")

        for one in all_ul:
            # 언론사
            com = one.find("a", class_="info press").text
            com_a = com.replace("언론사 선정", "")

            # print(one)
            tmp = one.find("a", class_="news_tit")

            article_n = tmp.text
            tmp_link = tmp.attrs['href']
            
            # 날짜
            date_one = one.find('div', class_='info_group').find("span", class_="info").text
            now_dates = date_cal(date_one)  # 날짜 계산
            
            print(article_n, com_a, now_dates)
            print("url : ", tmp_link)

            all_company1.append(com_a)
            all_article1.append(article_n)
            all_link1.append(tmp_link)
            all_date1.append(now_dates)
            
        print(len(all_article))
    return all_company1, all_article1, all_link1, all_date1

In [30]:
import pandas as pd
from datetime import datetime
import os

### 폴더 생성

In [31]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

try:
    if not os.path.exists(file_make_time):
        os.makedirs(file_make_time)
        print("folder creating ok")
except OSError:
    print("Error : Creating directory ", file_make_time)

In [32]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

all_company = []
all_article = []
all_link = []
all_date = []

for one in keyword_multi:
    url = 'https://search.naver.com/search.naver?query=' + one + '&where=news'
    driver.get(url)
    time.sleep(3)
    
    sel_search = driver.find_element_by_xpath('//*[@id="nx_query"]') 
    sel_btn = driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/button') 
    
    sel_search.clear()
    sel_search.send_keys(one)
    sel_btn.click()
    
    all_company, all_article, all_link, all_date = keyword_news_n(one, 
                                                                  file_make_time)
    
    dict_dat = {"언론사": all_company, "기사명" : all_article, 
                "날짜":all_date, "URL":all_link}
    
    dat = pd.DataFrame(dict_dat)
    dat['키워드'] = one
    dat.columns = ['키워드', "언론사", "기사명", "날짜", "URL"]
    
    fname_csv = file_make_time + "/" + one + "_" + file_make_time +".csv"
    fname_xlsx = file_make_time + "/" + one + "_" + file_make_time +".xlsx"
    dat.to_csv(fname_csv, index=False)
    dat.to_excel(fname_xlsx, index=False)

현재 페이지 :  1 키워드 :  고3 백신 접종 작성시간:  2021_1026_14
1
뉴스 날짜 :  2021-10-25
경남도 "도민 70% 코로나19 백신 접종 완료" 뉴시스 2021-10-25
url :  http://www.newsis.com/view/?id=NISX20211025_0001625985&cID=10812&pID=10800
뉴스 날짜 :  2021-10-25
유은혜 “소아·청소년 백신접종, 학교 정상화에 보탬 될 것” 이데일리 2021-10-25
url :  http://www.edaily.co.kr/news/newspath.asp?newsid=03411206629216528
위드코로나 앞두고 '학교 감염' 확산 우려…12~17세 접종률 0.6% 불과 뉴스1 2021-10-23
url :  https://www.news1.kr/articles/?4470035
국민 69.4% 코로나 백신접종 완료…70% 돌파 눈앞(종합) 뉴시스 2021-10-24
url :  http://www.newsis.com/view/?id=NISX20211023_0001624315&cID=10201&pID=10200
뉴스 날짜 :  2021-10-26
경남도민 70% 백신 접종 완료… 위드 코로나 추진 김해뉴스 2021-10-26
url :  http://www.gimhaenews.co.kr/news/articleView.html?idxno=43653
16~17세 청소년·임신부 백신 접종…"맞으니 안심" 연합뉴스TV 2021.10.18.
url :  http://www.yonhapnewstv.co.kr/MYH20211018009000641/?did=1825m
"생리불순 생기면 어떡하죠?"…청소년 코로나19 백신접종 Q&A 연합뉴스 2021-10-19
url :  http://yna.kr/AKR20211018127400530?did=1195m
코로나19 백신 접종 고3, 심근염·심낭염 10만 명당 3.6건 발생 KBS 2021-10-19
url :  https://

8
4분기 10대·임산부 백신접종 가속…고위험군 부스터샷 추진 뉴스핌 2021.09.27.
url :  http://www.newspim.com/news/view/20210927000683
고 1·2 백신 접종 시작…'경남 50% 사전 예약' 접종 vs 우려 LG헬로비전 2021-10-19
url :  http://news.lghellovision.net/news/newsView.do?soCode=SC20000000&idx=321336
고3 이어 초6~고2도…소아·청소년 백신 접종계획 오늘 발표 뉴스1 2021.09.27.
url :  https://www.news1.kr/articles/?4443845
12~17세 백신 접종 다음 달부터‥"접종 여부 자율 결정" MBC 2021.09.27.
url :  https://imnews.imbc.com/replay/2021/nwdesk/article/6303286_34936.html
16~17세 백신 예약 4시간 만에 19만명 참여…예약률 20.8% 뉴시스 2021.10.06.
url :  http://www.newsis.com/view/?id=NISX20211006_0001604656&cID=10201&pID=10200
고1·2 백신 예약 첫날 21%…"기대 이상, 고3 먼저 맞아 우려 줄어" 뉴스1 2021.10.06.
url :  https://www.news1.kr/articles/?4453790
고1~2, 내일부터 백신 사전예약…'미접종' 불이익 주나 뉴시스 2021.10.04.
url :  http://www.newsis.com/view/?id=NISX20211001_0001601025&cID=10201&pID=10200
美 텍사스주, '백신 의무화 금지' 행정명령... 바이든에 반기 한국일보 2021.10.12.
url :  https://hankookilbo.com/News/Read/A2021101210520002650?did=NA
'8만명 접종' 양천구 예방접종센터 10월말 문 닫는다 뉴스1 2021